In [1]:
import re
import collections
import nltk
nltk.download('punkt')
import sys
import numpy as np
import tensorflow as tf

[nltk_data] Downloading package punkt to C:\Users\gayan.DESKTOP-
[nltk_data]     3MLAI51\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
text_file = "corpus_100k.txt"

In [3]:
with open(text_file, 'r', encoding = 'utf-8') as f:
    text = f.read()

In [4]:
with open('corpus_100k_wr.txt', 'w') as f:
    f.write(text)

In [5]:
text[:100]

'Հիմա ես կարող է մի բառ մի քիչ անզգույշ արտահայտեմ , եւ երեք ամիս ՀՀ ներքաղաքական կյանքն այդպես էլ չխ'

In [6]:
def replace_punctuation(text):

  punctuation = re.compile(r"[^\w\s]")
  text = punctuation.sub(" ", text)

  text = text.replace("\n", " ")

  pattern = re.compile(r"\s+")
  text = pattern.sub(" ", text)

  return text

In [7]:
text = replace_punctuation(text)
text[:100]

'Հիմա ես կարող է մի բառ մի քիչ անզգույշ արտահայտեմ եւ երեք ամիս ՀՀ ներքաղաքական կյանքն այդպես էլ չխաղ'

In [8]:
def build_vocabulary(text, vocabulary_size):

  text = text.lower()
  tokens = nltk.word_tokenize(text)

  counter = collections.Counter(tokens)

  top_n_words = []
  for word, count in counter.most_common(vocabulary_size):
    word = word.strip()
    top_n_words.append(word)

  return top_n_words

In [9]:
vocabulary = build_vocabulary(text, 2500)

In [10]:
with open('vocab.txt', 'w') as f:
    f.write(str(vocabulary))

In [11]:
def create_training_dataset(word_sequence, window_size):

  skip_grams = []
  for i in range(1, len(word_sequence) - 1):
    input = word_sequence[i]
    context = []
    for j in range(-window_size, window_size + 1):
      if j != 0 and 0 <= i + j < len(word_sequence):
        context.append(word_sequence[i + j])

    for w in context:
      skip_grams.append([input, w])

  return skip_grams

In [12]:
window_size = 2
training_dataset = create_training_dataset(vocabulary, window_size)
training_dataset[:100]

[['որ', 'է'],
 ['որ', 'են'],
 ['որ', 'և'],
 ['են', 'է'],
 ['են', 'որ'],
 ['են', 'և'],
 ['են', 'եւ'],
 ['և', 'որ'],
 ['և', 'են'],
 ['և', 'եւ'],
 ['և', 'ի'],
 ['եւ', 'են'],
 ['եւ', 'և'],
 ['եւ', 'ի'],
 ['եւ', 'հետ'],
 ['ի', 'և'],
 ['ի', 'եւ'],
 ['ի', 'հետ'],
 ['ի', 'համար'],
 ['հետ', 'եւ'],
 ['հետ', 'ի'],
 ['հետ', 'համար'],
 ['հետ', 'այս'],
 ['համար', 'ի'],
 ['համար', 'հետ'],
 ['համար', 'այս'],
 ['համար', 'մասին'],
 ['այս', 'հետ'],
 ['այս', 'համար'],
 ['այս', 'մասին'],
 ['այս', 'ու'],
 ['մասին', 'համար'],
 ['մասին', 'այս'],
 ['մասին', 'ու'],
 ['մասին', 'այդ'],
 ['ու', 'այս'],
 ['ու', 'մասին'],
 ['ու', 'այդ'],
 ['ու', 'ին'],
 ['այդ', 'մասին'],
 ['այդ', 'ու'],
 ['այդ', 'ին'],
 ['այդ', 'էր'],
 ['ին', 'ու'],
 ['ին', 'այդ'],
 ['ին', 'էր'],
 ['ին', 'հհ'],
 ['էր', 'այդ'],
 ['էր', 'ին'],
 ['էր', 'հհ'],
 ['էր', 'իր'],
 ['հհ', 'ին'],
 ['հհ', 'էր'],
 ['հհ', 'իր'],
 ['հհ', 'այն'],
 ['իր', 'էր'],
 ['իր', 'հհ'],
 ['իր', 'այն'],
 ['իր', 'չի'],
 ['այն', 'հհ'],
 ['այն', 'իր'],
 ['այն', 'չի'],
 ['այն', 'թ

In [13]:
def random_batch(data, size, vocabulary):

  if size > len(data):
    raise ValueError("The size of the batch is greater than the size of the data.")

  random_index = np.random.choice(range(len(data)), size, replace=False)

  random_inputs = []
  random_labels = []
  for i in random_index:
    # one-hot encoding of words
    random_inputs.append(np.eye(len(vocabulary))[vocabulary.index(data[i][0])])  # input
    random_labels.append(data[i][1])  # context word

  return random_inputs, random_labels

In [14]:
random_inputs, random_labels = random_batch(training_dataset,100, vocabulary)